In [1]:
# Import dependencies
import pandas as pd 

In [ ]:
# Reference to CSV and reading CSV into Pandas DataFrame
csv_path = "Resources/BehaviorialRiskPrevention.csv"
behaviorialrisk_df = pd.read_csv(csv_path)
chocolate_ratings_df.head(10)

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [ ]:
spark = SparkSession \
    .builder \
    .appName('pyspark_demo_app') \
    .config('spark.driver.extraClassPath',
            'postgresql-42.2.9') \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://inspector20jan.s3.us-east-2.amazonaws.com/user_data.csv"
spark.sparkContext.addFile(url)

user_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("user_data.csv"), inferSchema=True, sep=',')
user_data_df.show(10)

In [ ]:
# Load in user_payment.csv from S3 into a DataFrame

url = "https://inspector20jan.s3.us-east-2.amazonaws.com/user_payment.csv"
spark.sparkContext.addFile(url)

user_payment_df = spark.read.option('header', 'true').csv(SparkFiles.get("user_payment.csv"), inferSchema=True, sep=',')
user_payment_df.show(10)

In [ ]:
# Join the two DataFrame
joined_df= user_data_df.join(user_payment_df, on="username", how="inner")
joined_df.show()

In [ ]:
# Drop null values
dropna_df = joined_df.dropna()
dropna_df.show()

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with active users
cleaned_df = dropna_df.filter(col("active_user")  == True)
cleaned_df.show()

In [ ]:
# Create user dataframe to match active_user table
clean_user_df = cleaned_df.select(["id", "first_name", "last_name", "username"])
clean_user_df.show()

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://heartsense.cgnk1yoo8fys.us-east-2.rds.amazonaws.com/heartsense"
config = {"user":"root", 
          "password": "Death4bid!", 
          "driver":"org.postgresql.Driver"}

In [ ]:
my_df = spark.read.jdbc(url=jdbc_url, table='doctors', properties=config)
my_df.limit(10).show()

In [ ]:
# Append DataFrame to active_user table in RDS

clean_user_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)